# Analysis of tweet text

In [1]:
import pymongo
import re
import pprint
import logging
from lark import Lark

In [2]:
# Logging functionality
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

2018-05-11 18:34:46,821 - DEBUG - Start of program


In [3]:
# Get the tweets stored in MongoDB
client = pymongo.MongoClient()
tweets = client['twitter']['tweets'].find()
latest_tweet = tweets[10]

In [4]:
def remove_from_word_list(original_list, list_subset):
    for each in list_subset:
        idx = original_list.index(each)
        del original_list[idx]
    return original_list

In [5]:
def print_important_content(tweet):
    print('By {} on {}\n'.format(tweet['user']['name'], tweet['created_at']))
    print('"{}"\n'.format(tweet['full_text']))
    if not tweet['entities']['urls']:
        expanded_url = 'No URL'
    else:
        expanded_url = tweet['entities']['urls'][0]['expanded_url']
    print('{}\n'.format(expanded_url))
    pprint.pprint(tweet['entities'])
    
print_important_content(latest_tweet)

By Kevin Rose ⛩ on Mon Apr 30 00:06:13 +0000 2018

"COM port 2 = IRQ 3."

No URL

{'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []}


In [6]:
def hashtag(tweet):
    hashtag_list = []
    for tag in tweet['hashtags']:
        hashtag_list.append('#' + tag['text'])
    return hashtag_list

hashtags = hashtag(latest_tweet['entities'])
hashtags

[]

In [7]:
latest_tweet

{'_id': ObjectId('5ae674b81c398f21b4830aa5'),
 'created_at': 'Mon Apr 30 00:06:13 +0000 2018',
 'id': 990744091443970048,
 'id_str': '990744091443970048',
 'full_text': 'COM port 2 = IRQ 3.',
 'truncated': False,
 'display_text_range': [0, 19],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 657863,
  'id_str': '657863',
  'name': 'Kevin Rose ⛩',
  'screen_name': 'kevinrose',
  'location': 'San Francisco, CA',
  'description': '',
  'url': 'https://t.co/MvhqAnT9jE',
  'entities': {'url': {'urls': [{'url': 'https://t.co/MvhqAnT9jE',
      'expanded_url': 'http://www.kevinrose.com/about',
      'display_url': 'kevinrose.com/about',
      'indices': [0, 23]}]},
   'description': {'urls': []}},
  'pro

In [8]:
# Break the text content apart for better formatting.
words = latest_tweet['full_text'].split(' ')

In [9]:
# Display the broken down list of each 'word'.
words

['COM', 'port', '2', '=', 'IRQ', '3.']

In [10]:
words = remove_from_word_list(words, hashtags)
words

['COM', 'port', '2', '=', 'IRQ', '3.']

In [11]:
# Find any urls in the list of words.
# Located better regex for url pattern matching - http://www.noah.org/wiki/RegEx_Python
def url_locator(word_list):
    urls = []
    for word in word_list:
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', word)
        if url:
            urls.append(url[0])
    return urls
urls = url_locator(words)
urls

[]

In [12]:
words = remove_from_word_list(words, urls)
words

['COM', 'port', '2', '=', 'IRQ', '3.']

In [13]:
# remove @mentions
def detect_user_handles(word_list):
    user_handle = []
    for word in word_list:
        if word.startswith('@'):
            user_handle.append(word)
    return user_handle

users = detect_user_handles(words)
users

[]

In [14]:
words = remove_from_word_list(words, users)
words

['COM', 'port', '2', '=', 'IRQ', '3.']

In [15]:
# def remove_extra_characters(word_list):
def strip_nonalphanumeric_characters(word_list):
    clean_list = []
    regex = re.compile('[^a-zA-Z0-9]')
    for word in word_list:
        word = regex.sub('', word)
        if word != '':
            clean_list.append(word)
    return clean_list
clean_words = strip_nonalphanumeric_characters(words)
clean_words

['COM', 'port', '2', 'IRQ', '3']

In [16]:
list(set(words) - set(clean_words))

['=', '3.']

In [17]:
clean_words=[x.lower() for x in clean_words]
clean_words.sort()
clean_words.sort(key=len)
clean_words

['2', '3', 'com', 'irq', 'port']

In [18]:
logging.debug('End of program')

2018-05-11 18:34:46,941 - DEBUG - End of program
